## Copyright 2023 Google LLC

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# MultiDiffusion with StyleAligned over SD v2

In [ ]:
import torch
from diffusers import StableDiffusionPanoramaPipeline, DDIMScheduler
import mediapy
import sa_handler
import pipeline_calls

In [ ]:
# init models
model_ckpt = "stabilityai/stable-diffusion-2-base"
scheduler = DDIMScheduler.from_pretrained(model_ckpt, subfolder="scheduler")
pipeline = StableDiffusionPanoramaPipeline.from_pretrained(
     model_ckpt, scheduler=scheduler, torch_dtype=torch.float16
).to("cuda")

sa_args = sa_handler.StyleAlignedArgs(share_group_norm=True,
                                      share_layer_norm=True,
                                      share_attention=True,
                                      adain_queries=True,
                                      adain_keys=True,
                                      adain_values=False,
                                     )
handler = sa_handler.Handler(pipeline)
handler.register(sa_args)

In [ ]:
# run MultiDiffusion with StyleAligned

reference_prompt = "a beautiful papercut art design"
target_prompts = ["mountains in a beautiful papercut art design", "giraffes in a beautiful papercut art design"]
view_batch_size = 25  # adjust according to VRAM size
reference_latent = torch.randn(1, 4, 64, 64,)
for target_prompt in target_prompts:
    images = pipeline_calls.panorama_call(pipeline, [reference_prompt, target_prompt], reference_latent=reference_latent, view_batch_size=view_batch_size)
    mediapy.show_images(images, titles=["reference", "result"])